In [1]:
import os
import pandas as pd
from PyPDF2 import PdfReader
import re

In [2]:
# Function 1: Extract title, text, and length
def extract_title_text_length(text):
    # Extract Title
    title = text.split('\n')[0]
    body_index = text.find('Body')
    article_text = text[body_index + len('Body'):].strip()
    
    # Try to find the length by looking for any characters after "Length:"
    length_match = re.search(r'Length:\s?(.+)', text)
    
    if length_match:
        length_str = length_match.group(1).strip()  # Extract the entire string and remove leading/trailing spaces
        # Check if the string contains only digits
        if length_str.isdigit():
            length = int(length_str)  # Convert to integer if it contains only digits
        else:
            length = length_str  # Keep as string if it contains other characters
    else:
        length = None

    return title, length

In [3]:
# Function 2: Extract newspaper name
def extract_newspaper_name_from_start(text, newspaper_list_de, lines_to_check=5):
    """
    Extracts the newspaper name from the start of a given text.

    Parameters:
    - text (str): The text from which to extract the newspaper name.
    - newspaper_list (list): A list of newspaper names to search for.
    - lines_to_check (int): The number of lines from the start of the text to check for the newspaper name.

    Returns:
    - str: The name of the newspaper if found, otherwise None.
    """
    newspaper_name = None  # Initialize the variable to hold the newspaper name
    lines = text.split('\n')[:lines_to_check]  # Split the text into lines and only consider the specified number of lines from the start

    # Iterate through each line, checking for the presence of any newspaper names
    for line in lines:
        for newspaper in newspaper_list_de:
            # Check if the newspaper name is in the current line, ignoring case
            if newspaper.lower() in line.lower():
                newspaper_name = newspaper  # Assign the newspaper name if found
                return newspaper_name  # Return immediately upon finding a newspaper name

    return newspaper_name  # Return the found newspaper name or None if not found


# List of major German newspapers
newspaper_list_de = [
    'Bild',
    'Süddeutsche Zeitung',
    'Frankfurter Allgemeine Zeitung',
    'Die Welt',
    'Handelsblatt',
    'Die Zeit',
    'Die ZEIT',
    'Frankfurter Rundschau',
    'Der Tagesspiegel',
    'Die Tageszeitung',
    'taz, die tageszeitung',
    'Berliner Morgenpost',
    'Stuttgarter Zeitung',
    'Stuttgarter Nachrichten',
    'Hamburger Abendblatt',
    'Abendzeitung',
    'Münchner Merkur',
    'Westdeutsche Allgemeine Zeitung',
    'Rheinische Post',
    'Kölner Stadt-Anzeiger',
    'Express',
    'Neue Osnabrücker Zeitung',
    'Augsburger Allgemeine',
    'Berliner Zeitung',
    'Badische Zeitung',
    'B.Z.',
    'Lübecker Nachrichten'
]

In [4]:
# Function 3: Extract and convert the date
def extract_date_de(text):
    # Mapping of German days and months to English
    day_mapping = {
        'Montag': 'Monday', 'Dienstag': 'Tuesday', 'Mittwoch': 'Wednesday', 
        'Donnerstag': 'Thursday', 'Freitag': 'Friday', 'Samstag': 'Saturday', 
        'Sonntag': 'Sunday'
    }
    month_mapping = {
        'Januar': 'January', 'Februar': 'February', 'März': 'March', 'April': 'April',
        'Mai': 'May', 'Juni': 'June', 'Juli': 'July', 'August': 'August',
        'September': 'September', 'Oktober': 'October', 'November': 'November', 'Dezember': 'December'
    }

    # Define date pattern in German
    date_pattern = r'(\b(?:Montag|Dienstag|Mittwoch|Donnerstag|Freitag|Samstag|Sonntag)?,?\s?\d{1,2}\. (Januar|Februar|März|April|Mai|Juni|Juli|August|September|Oktober|November|Dezember) \d{4})'

    date_match = re.search(date_pattern, text, re.IGNORECASE)
    date = None

    if date_match:
        date_str = date_match.group(0)
        # Replace German days and months with English
        for de, en in day_mapping.items():
            date_str = re.sub(de, en, date_str, flags=re.IGNORECASE)
        for de, en in month_mapping.items():
            date_str = re.sub(de, en, date_str, flags=re.IGNORECASE)

        # Attempt to parse the date string into a datetime object
        try:
            date = pd.to_datetime(date_str, errors='coerce', dayfirst=True)
            if date is not None and not pd.isnull(date):
                date = date.strftime('%Y-%m-%d')
            else:
                date = date_match.group(0)  # Fallback: Return the original extracted date string
        except ValueError:
            date = date_match.group(0)  # Fallback: Return the original extracted date string

    return date

In [5]:
#Function 4: Extract text correctly without additional "Load-Date" etc
def extract_text_from_pdf(pdf_path):
    text = ''
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() if page.extract_text() else ''
    return text

start_indicator = "Body"
end_indicator = "Load-Date"

def identify_text_body(text, start_indicator, end_indicator):
    start_index = text.find(start_indicator)
    if start_index != -1:
        start_index += len(start_indicator)  # Move start index to the end of the start_indicator
    else:
        start_index = 0  # Start from the beginning if start_indicator not found
    
    end_index = text.find(end_indicator, start_index) if end_indicator in text and start_index != -1 else len(text)
    
    if start_index != -1 and end_index != -1:
        return text[start_index:end_index].strip()
    else:
        return text[start_index:].strip()  # Return the text from start_indicator to the end if end_indicator is not found

In [6]:
# Function 5: Identify the language
def identify_language(folder_path):
    """
    Identifies the language based on markers in the folder path.

    Parameters:
    - folder_path (str): The path of the folder which likely contains language markers.

    Returns:
    - str: The identified language ('French', 'German', or 'Unknown').
    """
    if "FR" in folder_path:
        return 'French'
    elif "DE" in folder_path:
        return 'German'
    else:
        return 'Unknown'

In [9]:
def compile_and_save_corpus_de(directory_path, output_xlsx_path):
    corpus = []
    file_count = 0  # To keep track of how many files have been processed

    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            file_path = os.path.join(directory_path, filename)
            print(f"Processing file: {filename}")  # Debugging statement

            text = extract_text_from_pdf(file_path)
            title, length = extract_title_text_length(text)
            article_text = identify_text_body(text, start_indicator, end_indicator)
            date = extract_date_de(text)
            newspaper_name = extract_newspaper_name_from_start(text, newspaper_list_de, lines_to_check=7)
            language = identify_language(directory_path)

            corpus.append({
                'Title': title,
                'Text Body': article_text,
                'Length': length,
                'Date': date,
                'Newspaper Name': newspaper_name,
                'Language': language
            })

            file_count += 1
            print(f"File processed: {filename}")  # Debugging statement

    print(f"Total files processed: {file_count}")  # Debugging statement

    corpus_df = pd.DataFrame(corpus)
    writer = pd.ExcelWriter(output_xlsx_path, engine='openpyxl')
    corpus_df.to_excel(writer, index=False)
    writer.save()
    print(f"Corpus saved to: {output_xlsx_path}")  # Debugging statement

In [10]:
# Execute the function
directory_path = 'C:\\Users\\eugen\\corpus_article_debt\\raw files\\raw docs DE'
output_xlsx_path = 'C:\\Users\\eugen\\corpus_article_debt\\corpus_article_debt_DE.xlsx'
compile_and_save_corpus_de(directory_path, output_xlsx_path)

Processing file: 'Wirtschaftsweise' Schnitzer_ Dividende oder Staatshilfe.pdf
File processed: 'Wirtschaftsweise' Schnitzer_ Dividende oder Staatshilfe.pdf
Processing file: ,,Da kann einem schwindlig werden_.pdf
File processed: ,,Da kann einem schwindlig werden_.pdf
Processing file: ,,Daf_r gibt es kein Drehbuch' In einer historischen Sitzung verabschieden die Abgeordneten des Bund.pdf
File processed: ,,Daf_r gibt es kein Drehbuch' In einer historischen Sitzung verabschieden die Abgeordneten des Bund.pdf
Processing file: ,,Das ist keine BLASE_ Der Plan f_r Ihr GELD 2024. Aktien, Anleihen, Zinsen, Inflation_ F_nf Top-Inv.pdf
File processed: ,,Das ist keine BLASE_ Der Plan f_r Ihr GELD 2024. Aktien, Anleihen, Zinsen, Inflation_ F_nf Top-Inv.pdf
Processing file: ,,Das ist schon ein ordentliches Packerl_.pdf
File processed: ,,Das ist schon ein ordentliches Packerl_.pdf
Processing file: ,,Den M_rchenwald gibt es gar nicht_ Philipp zu Guttenberg ist Pr_sident der deutschen Waldbesitzer.pdf
Fi

File processed: Als g_be es kein Morgen.pdf
Processing file: Als w_re nichts geschehen Italien f_hlt sich um den Lohn der Reformen betrogen und von Anlegern verk.pdf
File processed: Als w_re nichts geschehen Italien f_hlt sich um den Lohn der Reformen betrogen und von Anlegern verk.pdf
Processing file: Alte Lasten und neue Akzente.pdf
File processed: Alte Lasten und neue Akzente.pdf
Processing file: Am l_ngeren Hebel.pdf
File processed: Am l_ngeren Hebel.pdf
Processing file: Andere Zeiten, andere Reden Atomausstieg und Griechenlands Euro-Beitritt - die Bundestagsdebatte vom.pdf
File processed: Andere Zeiten, andere Reden Atomausstieg und Griechenlands Euro-Beitritt - die Bundestagsdebatte vom.pdf
Processing file: ANGELA MERKEL _Es muss auch Vergebung geben_.pdf
File processed: ANGELA MERKEL _Es muss auch Vergebung geben_.pdf
Processing file: Angriff auf die Allmacht des Dollars Mit dem Renminbi k_nnte der Welt bald eine neue Leitw_hrung bev.pdf
File processed: Angriff auf die Allmacht 

File processed: Beim Sparen lieber klotzen statt kleckern.pdf
Processing file: BENELUX.pdf
File processed: BENELUX.pdf
Processing file: Berlin will Vorbild beim Sparen sein - Tagesthema.pdf
File processed: Berlin will Vorbild beim Sparen sein - Tagesthema.pdf
Processing file: Berliner Blockade.pdf
File processed: Berliner Blockade.pdf
Processing file: Berlusconis Regierung gewinnt Vertrauensabstimmung.pdf
File processed: Berlusconis Regierung gewinnt Vertrauensabstimmung.pdf
Processing file: Bertelsmann-Studie_ Schuldenbremse trotz h_herer Investitionen.pdf
File processed: Bertelsmann-Studie_ Schuldenbremse trotz h_herer Investitionen.pdf
Processing file: Besorgte Manager am Telefon Die T_rkei galt als Musterbeispiel eines erfolgreichen Schwellenlands. D.pdf
File processed: Besorgte Manager am Telefon Die T_rkei galt als Musterbeispiel eines erfolgreichen Schwellenlands. D.pdf
Processing file: Besorgte Unternehmer am Telefon Die T_rkei galt als Musterbeispiel eines erfolgreichen Schwel

File processed: China ist auch nur ein Dominostein Die Volksrepublik hat trotz St_rken das Potenzial, die n_chste We.pdf
Processing file: CHRISTIAN LINDNER _Gespenstische Politik in der Euro-Krise_.pdf
File processed: CHRISTIAN LINDNER _Gespenstische Politik in der Euro-Krise_.pdf
Processing file: CHRISTIAN LINDNER _Telekom-Aktien f_r Glasfaser-Infrastruktur_.pdf
File processed: CHRISTIAN LINDNER _Telekom-Aktien f_r Glasfaser-Infrastruktur_.pdf
Processing file: CHRISTOS STAIKOURAS _Wir kennen unsere Schw_chen_.pdf
File processed: CHRISTOS STAIKOURAS _Wir kennen unsere Schw_chen_.pdf
Processing file: CHRONIK _Mehr Geld l_st nicht die Probleme_ FDP-Fraktionschef Rainer Br_derle _ber Schulden, Zinsen.pdf
File processed: CHRONIK _Mehr Geld l_st nicht die Probleme_ FDP-Fraktionschef Rainer Br_derle _ber Schulden, Zinsen.pdf
Processing file: CHRONIK _Wir m_ssen f_r alle da sein_ Bremens B_rgermeister Jens B_hrnsen _ber das Profil der SPD, d.pdf
File processed: CHRONIK _Wir m_ssen f_r alle da

File processed: Dem Tod entronnen.pdf
Processing file: Demokraten sind keine Schwaben.pdf
File processed: Demokraten sind keine Schwaben.pdf
Processing file: Den Abstand zu China wird Europa nicht verringern k_nnen Die Wunden der Euro-Krise von 2012 sind nic.pdf
File processed: Den Abstand zu China wird Europa nicht verringern k_nnen Die Wunden der Euro-Krise von 2012 sind nic.pdf
Processing file: Der angepasste Liberalismus.pdf
File processed: Der angepasste Liberalismus.pdf
Processing file: Der Anwalt der Bundeskanzlerin.pdf
File processed: Der Anwalt der Bundeskanzlerin.pdf
Processing file: Der bequeme Ausweg.pdf
File processed: Der bequeme Ausweg.pdf
Processing file: Der blinde Passagier.pdf
File processed: Der blinde Passagier.pdf
Processing file: DER CHEF_KONOM Das Handicap der n_chsten Regierung.pdf
File processed: DER CHEF_KONOM Das Handicap der n_chsten Regierung.pdf
Processing file: DER CHEF_KONOM Die undemokratische Supermacht.pdf
File processed: DER CHEF_KONOM Die undemokra

...

File processed: Wiederwahl gef_hrdet - aber nicht ausgeschlossen.pdf
Processing file: Willk_rliche - Defizitgrenzen -.pdf
File processed: Willk_rliche - Defizitgrenzen -.pdf
Processing file: Wir haben jetzt eine Perspektive GELD Durch die Neuordnung der Bund-L_nder-Finanzen kann sich Bremen.pdf
File processed: Wir haben jetzt eine Perspektive GELD Durch die Neuordnung der Bund-L_nder-Finanzen kann sich Bremen.pdf
Processing file: Wir leben nicht _ber unsere Verh_ltnisse! Warum es falsch ist, den Sozialstaat f_r die Staatsschulde.pdf
File processed: Wir leben nicht _ber unsere Verh_ltnisse! Warum es falsch ist, den Sozialstaat f_r die Staatsschulde.pdf
Processing file: Wir m_ssen die Armut stoppen Internationale NGOs gingen bei CIVIL20 hart mit den G-20-Regierungen in.pdf
File processed: Wir m_ssen die Armut stoppen Internationale NGOs gingen bei CIVIL20 hart mit den G-20-Regierungen in.pdf
Processing file: Wir reden hier von richtig viel Geld Schuldenfrei werden wir alle zu Lebzeiten n

File processed: Zwischen Korea und Karlsruhe - die Kanzlerin im Interview Bundeskanzlerin Angela Merkel im Interview.pdf
Processing file: Z_hneknirschend gl_cklich Die SPD entdeckt die soziale Frage neu. Doch ideologische Ideen zwischen S.pdf
File processed: Z_hneknirschend gl_cklich Die SPD entdeckt die soziale Frage neu. Doch ideologische Ideen zwischen S.pdf
Processing file: _ F_r Entlassungen gibt es keinen Grund_ Verdi-Chef Werneke _ber die enorme Not der Besch_ftigten un.pdf
File processed: _ F_r Entlassungen gibt es keinen Grund_ Verdi-Chef Werneke _ber die enorme Not der Besch_ftigten un.pdf
Processing file: _ Merkels Plan ist unrealistisch_ FINANZEN Der Versuch, die europ_ische Lohn- und Rentenpolitik zu v.pdf
File processed: _ Merkels Plan ist unrealistisch_ FINANZEN Der Versuch, die europ_ische Lohn- und Rentenpolitik zu v.pdf
Processing file: _Als Erstes m_ssen wir investieren_ Kein Finanzsenator hat je so viele Schulden aufgenommen wie Andr.pdf
File processed: _Als Erstes 

File processed: _Die Gr_nen haben viel erreicht_.pdf
Processing file: _Die Gr_nen haben viel erreicht__Attachment1.pdf
File processed: _Die Gr_nen haben viel erreicht__Attachment1.pdf
Processing file: _Die Klimaschutzbewegung ist nach links gekippt_ Umweltaktivist Heinrich Str__enreuther will die CDU.pdf
File processed: _Die Klimaschutzbewegung ist nach links gekippt_ Umweltaktivist Heinrich Str__enreuther will die CDU.pdf
Processing file: _Die Krise zeigt die Schwachstelle im Sozialsystem_ DGB-Chef Reiner Hoffmann fordert weitreichende Z.pdf
File processed: _Die Krise zeigt die Schwachstelle im Sozialsystem_ DGB-Chef Reiner Hoffmann fordert weitreichende Z.pdf
Processing file: _Die Nachhaltigkeit der Geldpolitik ist ein Etikettenschwindel_ Die Europ_ische Zentralbank habe den.pdf
File processed: _Die Nachhaltigkeit der Geldpolitik ist ein Etikettenschwindel_ Die Europ_ische Zentralbank habe den.pdf
Processing file: _Die politischen Lager geh_ren der Vergangenheit an_ Staatsverschuldun

File processed: _In diesem Jahr schalten wir auf Angriff_ Vorstandssprecher von Roland Berger erwartet wachsende Nac.pdf
Processing file: _In Wahljahren sitzt das Geld grunds_tzlich etwas lockerer_ Der Ex-Chef der Wirtschaftsweisen _ber d.pdf
File processed: _In Wahljahren sitzt das Geld grunds_tzlich etwas lockerer_ Der Ex-Chef der Wirtschaftsweisen _ber d.pdf
Processing file: _Italien ist ein solides Land_ Au_enminister Paolo Gentiloni klagt an_ Br_ssel gebe den Schulmeister.pdf
File processed: _Italien ist ein solides Land_ Au_enminister Paolo Gentiloni klagt an_ Br_ssel gebe den Schulmeister.pdf
Processing file: _Italienische Weine werden getrunken, nicht gesammelt_.pdf
File processed: _Italienische Weine werden getrunken, nicht gesammelt_.pdf
Processing file: _Jeder Mensch kann Europa ver_ndern_ Ferdinand von Schirach, derzeit erfolgreichster Schriftsteller.pdf
File processed: _Jeder Mensch kann Europa ver_ndern_ Ferdinand von Schirach, derzeit erfolgreichster Schriftsteller.pdf
P

File processed: _Willst du das, Hans-Werner__ Deutschlands prominenteste _konomen streiten, ob die steigenden Staats.pdf
Processing file: _Wir brauchen deutlich h_here L_hne_ Hamburgs B_rgermeister Olaf Scholz will seine Partei mit wenig.pdf
File processed: _Wir brauchen deutlich h_here L_hne_ Hamburgs B_rgermeister Olaf Scholz will seine Partei mit wenig.pdf
Processing file: _Wir erkennen Potenzial bei Value_ CIO_ Aktien bieten attraktive Ertragschancen - Ausgew_hlte Untern.pdf
File processed: _Wir erkennen Potenzial bei Value_ CIO_ Aktien bieten attraktive Ertragschancen - Ausgew_hlte Untern.pdf
Processing file: _Wir Europ_er leben im Adelsstand_ Milo Rau ist derzeit der einflussreichste Regisseur des Kontinent.pdf
File processed: _Wir Europ_er leben im Adelsstand_ Milo Rau ist derzeit der einflussreichste Regisseur des Kontinent.pdf
Processing file: _Wir haben Inflation vor der T_r_ Aufsichtsrat erwartet steigende Zinsen, die den Akteuren an den Fi.pdf
File processed: _Wir haben Inf